In [5]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import StreamingResponse
from pydantic import BaseModel
import asyncio
import json

# Import your modified run_finance_agent function
from your_finance_agent_module import run_finance_agent

app = FastAPI()

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class QueryInput(BaseModel):
    query: str

async def event_generator(query):
    async for state in run_finance_agent(query):
        yield f"data: {json.dumps(state)}\n\n"
    yield "data: [DONE]\n\n"

@app.post("/query")
async def process_query(query_input: QueryInput):
    return StreamingResponse(event_generator(query_input.query), media_type="text/event-stream")

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
import streamlit as st
import asyncio
from typing import AsyncGenerator
import json
from your_finance_agent_module import run_finance_agent  # Import your finance agent function

# Streamlit app layout
st.title("Finance Agent Query Interface")

# Input for user query
query = st.text_input("Enter your query:")

# Submit button
if st.button("Submit"):
    if query:
        # Create a placeholder for the graph
        graph_placeholder = st.empty()
        
        # Create a placeholder for the results
        results_placeholder = st.empty()

        # Function to update the graph visualization
        def update_graph(current_node: str):
            # This is a simple text-based representation of the graph
            # You can replace this with a more sophisticated graph visualization library if needed
            nodes = ["subgraph_router", "execute_subgraphs", "aggregate_results", "end"]
            graph = ""
            for node in nodes:
                if node == current_node:
                    graph += f"[{node}] -> "
                else:
                    graph += f"{node} -> "
            graph = graph[:-4]  # Remove the last arrow
            graph_placeholder.text(f"Current State: {graph}")

        # Run the finance agent and update the UI
        async def run_query():
            try:
                async for state in run_finance_agent(query):
                    # Update the graph
                    update_graph(state.get('current_node', ''))
                    
                    # Update the results
                    results_placeholder.json(state)
                    
                    # Add a small delay to make the updates visible
                    await asyncio.sleep(0.1)
                
                st.success("Query processing completed!")
            except Exception as e:
                st.error(f"An error occurred: {str(e)}")

        # Run the async function
        asyncio.run(run_query())
    else:
        st.warning("Please enter a query.")

# Add some information about how to use the app
st.markdown("""
## How to use this app:
1. Enter your finance-related query in the text box above.
2. Click the 'Submit' button to process your query.
3. Watch the graph update to show the current state of processing.
4. View the detailed results in the JSON output below the graph.
""")